## Load/import packages

In [1]:
import ffmpeg
import time
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, exposure, color
from skimage.transform import resize

# Import MTCNN for face detection (method 1)
# from mtcnn.mtcnn import MTCNN

# Import MTCNN for face detection and alignment (method 2)
from deepface import DeepFace

from os import listdir, mkdir
from os.path import splitext
from os.path import join

import tensorflow as tf

%matplotlib inline

tf.__version__

Using TensorFlow backend.
C:\Users\ion\anaconda3\envs\tf-gpu-2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ion\anaconda3\envs\tf-gpu-2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ion\anaconda3\envs\tf-gpu-2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ion\anaconda3\envs\tf-gpu-2\lib\sit

'1.14.0'

## Directories
### Aff-Wild2

In [2]:
from load_filenames import (AF7_dir_videos, 
                            AF7_dir_labels, 
                            AF7_labeled_videos_FN)

# Start Pre-processing

In [18]:
#assign the MTCNN detector
detector = MTCNN()

def extract_face(pixels):              
    f = detector.detect_faces(pixels)
    # If the face is not found return the empty list
    if not f:
        return f
    #fetching the (x,y)co-ordinate and (width-->w, height-->h) of the image
    x1,y1,w,h = f[0]['box']             
    x1, y1 = abs(x1), abs(y1)
    x2 = abs(x1+w)
    y2 = abs(y1+h)
    
    #locate the co-ordinates of face in the image
    face = pixels[y1:y2,x1:x2]
    face = resize(face, (112,112,3))
    return face

# Only face detection with MTCNN

In [22]:
# Define directory to save the frames
frame_dir = r'D:\AFEW 7.0 Dataset\Val+train_faces'

for label in AF7_dir_labels:
    try:
        mkdir(join(frame_dir, label))
    except FileExistsError:
        print(f"{label} Directory already exists!")
    
    videos = AF7_labeled_videos_FN.get(label)
    for video in videos:
        fn, ext = splitext(video)
        
        video_dir = join(join(frame_dir, label) , video)
        probe = ffmpeg.probe(join(AF7_dir_videos, join(label, video)))
        video_info = next(x for x in probe["streams"] if x["codec_type"] == "video")
        width = int(video_info["width"])
        height = int(video_info["height"])

        out, _ = (
            ffmpeg.input(join(AF7_dir_videos, join(label, video)))
            .output("pipe:", format="rawvideo", pix_fmt="rgb24")
            .run(capture_stdout=True)
        )
        frames = np.frombuffer(out, np.uint8).reshape([-1, height, width, 3])
        
        frame_n = 0
        for frame in frames:
            frame_n +=1
            face = extract_face(frame)
            # If the face is not found, continue in the loop
            if face == []:
                continue
            io.imsave(join(join(frame_dir, label), f'{fn}_{frame_n:05n}'+'.jpg'), (face*255).astype('uint8'), check_contrast=False)
        
                

C:\Users\ion\anaconda3\envs\tf-gpu-2\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


# Face detection with MTCCN and also face alignment

In [6]:
# # Define directory to save the frames
# frame_dir = r'D:\AFEW 7.0 Dataset\Val+train_faces_aligned'

# for label in AF7_dir_labels:
#     try:
#         mkdir(join(frame_dir, label))
#     except FileExistsError:
#         print(f"{label} Directory already exists!")
    
#     videos = AF7_labeled_videos_FN.get(label)
#     for video in videos:
#         fn, ext = splitext(video)
        
#         video_dir = join(join(frame_dir, label) , video)
#         probe = ffmpeg.probe(join(AF7_dir_videos, join(label, video)))
#         video_info = next(x for x in probe["streams"] if x["codec_type"] == "video")
#         width = int(video_info["width"])
#         height = int(video_info["height"])

#         out, _ = (
#             ffmpeg.input(join(AF7_dir_videos, join(label, video)))
#             .output("pipe:", format="rawvideo", pix_fmt="rgb24")
#             .run(capture_stdout=True)
#         )
#         frames = np.frombuffer(out, np.uint8).reshape([-1, height, width, 3])
        
#         frame_n = 0
#         for frame in frames:
#             frame_n +=1
#             try:
#                 face = DeepFace.detectFace(frame, detector_backend = 'ssd', enforce_detection=False)
#                 face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
#                 face = resize(face, (112,112,3))
#             except FileExistsError:
#                 print("no face")

#             print(f'{fn}_{frame_n:05n}'+'.jpg')
#             try:
#                 io.imsave(join(join(frame_dir, label), f'{fn}_{frame_n:05n}'+'.jpg'), (face*255).astype('uint8'), check_contrast=False)
#             except FileExistsError:
#                 print("no face to save")

Angry Directory already exists!


TypeError: detectFace() got an unexpected keyword argument 'enforce_detection'

*** 